<a href="https://colab.research.google.com/github/Aditya-1301/ToonCrafter_Experiment/blob/main/init_tc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>